In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:90% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:12pt;}
div.text_cell_render.rendered_html{font-size:12pt;}
div.output {font-size:12pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:12pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:5px;}
table.dataframe{font-size:12px;}
</style>
"""))

# <span style="color:red">ch9.09_chroma→pinecone을 활용한 RAG활용(upstage Embedding)</span>

# 벡터DB : Chroma vs. Pinecone

- chroma : 인메모리 vector DB , 로컬 vector DB
- Pinecone : 클라우드 vector DB
   (https://www.pinecone.io/에서 api key생성 -> .env에 추가(PINECONE_API_KEY등록)

# 0. 패키지 설치

In [ ]:
%pip install -q pinecone langchain-pinecone --no-warn-script-location

# 1. knowledge Base 구성을 위한 데이터 생성

In [2]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
loader = Docx2txtLoader('data/소득세법(법률)(제21065호)(20260102).docx')
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
    #separators=["\n\n","\n"," ",""]
)
document_list = loader.load_and_split(text_splitter=text_splitter)
len(document_list)

193

In [3]:
# embedding : upstage의 solar-embedding-1-large-passage
from dotenv import load_dotenv
from langchain_upstage import UpstageEmbeddings
load_dotenv()
embedding = UpstageEmbeddings(model="solar-embedding-1-large-passage")

In [5]:
len(embedding.embed_query("소득세법"))

4096

In [7]:
%%time
# pinecone vector database 저장
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore
import os
pc = Pinecone(
    api_key=os.getenv("PINECONE_API_KEY")
)
# 데이터를 처음 업로드할 때 
# index_name = "tax-index-upstage"
# database = PineconeVectorStore.from_documents(
#     documents=document_list,
#     embedding=embedding,
#     index_name=index_name
# )
# 업로드시 경고가 안보이려면 아나콘다 프롬프트 llm환경에서 conda install -c conda-forge ipywidgets
# 업로드한 벡터DB를 가져올 때
database = PineconeVectorStore(
    embedding=embedding, # 질문을 임베딩하여 유사도 검색
    index_name=index_name
)

CPU times: total: 0 ns
Wall time: 5.13 ms


# 2. 답변 생성을 위한 Retrieval 

In [8]:
query = "연봉이 5천만원인 직장인의 소득세는 얼마인가요?"
retrieved_docs =database.similarity_search(query,k=3) # 기본k값 : 4

In [9]:
# retrieved_docs[1].page_content
retrieved_doc = "\n\n---\n\n".join([doc.page_content for doc in retrieved_docs])
#retrieved_doc

In [10]:
# 답변을 잘 모를때는 아래와 같이 (질문 알때는 위와 같이 retrieved_doc  ) 
retriever = database.as_retriever(
    search_kwargs={"k":3}
)
retrieved_docs=retriever.invoke(query)
retrieved_doc = "\n\n---\n\n".join([doc.page_content for doc in retrieved_docs])

# 3. 답변 생성

In [11]:
from langchain_openai import ChatOpenAI
llm=ChatOpenAI(model="gpt-4.1-nano")

In [12]:
# upstage에서 받은 20$로 llm을 사용하고 싶다면
from langchain_upstage import ChatUpstage
llm=ChatUpstage(
    model="solar-pro2",
    reasoning_effort="high"   # 느리지만 더 깊게 추론함(low,medium)
)


In [13]:
prompt = f"""[identity]
- 당신은 최고의 한국 소득세법 전문가입니다
- [context]를 참고해서 사용자의 질문에 답변해 주세요.
- [context]는 다음과 같아요
{retrieved_doc}
- 질문 : {query}"""

In [14]:
ai_message = llm.invoke(prompt)

In [15]:
print(ai_message.content)

연봉 5천만원인 직장인의 소득세 계산은 다음과 같이 진행됩니다:

### 1. **총급여액**
   - **50,000,000원**

### 2. **근로소득공제 적용**
   - **공제액 계산**:
     - 500만원 이하: 70% 공제 → 350만원  
     - 500만원 초과 ~ 1,500만원: 40% 공제 → (1,500 - 500) × 40% = 400만원  
     - 1,500만원 초과 ~ 4,500만원: 15% 공제 → (4,500 - 1,500) × 15% = 450만원  
     - 4,500만원 초과 ~ 5,000만원: 5% 공제 → (5,000 - 4,500) × 5% = 25만원  
     - **총 공제액**: 350 + 400 + 450 + 25 = **1,225만원**  
   - **공제 후 과세표준**:  
     50,000,000 - 12,250,000 = **37,750,000원**

### 3. **종합소득산출세액 계산**
   - **세율 적용**:
     - 1,400만원 이하: 6% → 84만원  
     - 1,400만원 초과 ~ 37,750,000원: 15% - 108만원  
       → (37,750,000 × 15%) - 1,080,000 = 5,662,500 - 1,080,000 = **4,582,500원**  
   - **산출세액**: **4,582,500원**

### 4. **세액공제 적용**
   - **근로소득세액공제** (제59조):
     - 총급여액 3,300만원 초과 ~ 7,000만원 이하 구간  
     → 74만원 - [(5,000 - 3,300) × 0.8%] = 740,000 - 13,600 = **726,400원**  
     - 최소 공제액 66만원 적용 → **660,000원**  
   - **공제 후 세액**:  
     4,582,500 - 660,000 = **3,922,500원**

### 5. **지방소득세 추가**
   - 국세의 10

# 4. langchain 답변 생성
- ch9.07_LangChain과vectorDatabase을 활용한 RAG구현(UpstageEmbedding) 참조

In [ ]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from dotenv import load_dotenv

# 1. LLM과 임베딩 초기화
load_dotenv()
#llm = ChatOpenAI(model="gpt-4.1-mini")
from langchain_upstage import ChatUpstage
llm = ChatUpstage(model="solar-pro2")

embedding = UpstageEmbeddings(model="solar-embedding-1-large-passage")
# 2. vector store load
vectorstore = Chroma(
    embedding_function=embedding,
    collection_name="tax-collection",
    persist_directory="./chroma_upstage/"
)
# 3. Retriever 생성
retriever = vectorstore.as_retriever(search_type="similarity",search_kwargs={"k":4})
# vectorstore.similarity_search("질문",k=2)  질문을 모르기때문에 사용 불가

# 4. 프롬프트 템플릿
template = f"""당신은 최고의 한국 소득세 전문가입니다.
다음 문맥을 참고하여 질문에 답하세요.
답을 모르면 모른다고 답하세요.
최대 3문장을 간결하게 답변하세요.
질문 : {{query}}
문맥 : {{context}}
답변 :"""
prompt = ChatPromptTemplate.from_template(template)
# 5. 검색된 document를 텍스트로 변환하는 함수
def format_documents(documents):
    return "\n\n---\n\n".join([doc.page_content for doc in documents])